In [ ]:
# USER: Change the path below to point to your local copy of ImageD11:

import os

home_dir = !echo $HOME
home_dir = str(home_dir[0])

# USER: You can change this location if you want

id11_code_path = os.path.join(home_dir, "Code/ImageD11")

import sys

sys.path.insert(0, id11_code_path)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

import scipy.ndimage as ndi

from ImageD11.nbGui import nb_utils as utils
import ImageD11.sinograms.dataset

from ImageD11.grain import grain
from ImageD11 import unitcell
%matplotlib ipympl

In [ ]:
# NOTE: For old datasets before the new directory layout structure, we don't distinguish between RAW_DATA and PROCESSED_DATA

### USER: specify your experimental directory

rawdata_path = "/data/visitor/ihma439/id11/20231211/RAW_DATA"

!ls -lrt {rawdata_path}

### USER: specify where you want your processed data to go

processed_data_root_dir = "/data/visitor/ihma439/id11/20231211/PROCESSED_DATA/James/nb_testing"

In [ ]:
# USER: pick a sample and a dataset you want to segment

sample = "FeAu_0p5_tR_nscope"
dataset = "top_100um"

In [ ]:
# desination of H5 files

dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")

In [ ]:
# Load the dataset
ds = ImageD11.sinograms.dataset.load(dset_path)

In [ ]:
ds.pbpfile = os.path.join(ds.analysispath, ds.dsname + '_pbp_index.pbp')

In [ ]:
# define your reference unit cell for RGB colour plotting

cf_2d = ds.get_cf_2d_from_disk()
cf_2d.parameters.loadparameters(ds.parfile)

cf_pars = cf_2d.parameters.get_parameters()
spacegroup = 229  # spacegroup for BCC iron
cf_pars["cell_lattice_[P,A,B,C,I,F,R]"] = spacegroup

ref_ucell = ImageD11.unitcell.unitcell_from_parameters(cf_pars)

In [ ]:
# pbp_array = np.loadtxt('/home/esrf/james1997a/Code/ImageD11/ImageD11/nbGui/S3DXRD/grainsout_all.pbp')

pbp_array = np.loadtxt(ds.pbpfile)
ubi = {}
i,j,n,u,ubi[0,0],ubi[0,1],ubi[0,2],ubi[1,0],ubi[1,1],ubi[1,2],ubi[2,0],ubi[2,1],ubi[2,2] = pbp_array.T
i = i.astype(int) 
i -= i.min()   # position in array / space
j = j.astype(int) 
j -= j.min()
n = n.astype(int)         # total peaks indexing with hkl==int with 0.03
u = u.astype(int)         # unique (h,k,l) labels on indexed peaks

In [ ]:
NY = int(i.max()-i.min())+1
i.min(),i.max(),NY

In [ ]:
fig, ax = plt.subplots()
ax.hist(u, bins=128)
ax.set_xlabel('Unique spots per pixel')
ax.set_ylabel('Count')
plt.show()

In [ ]:
minpeaks = 50

uni = np.ones((NY,NY))
uni.fill(minpeaks) # peak cutorr
npk = np.zeros((NY,NY))
ubifit = np.zeros((NY,NY,3,3), float)
ubifit.fill(np.nan)
ubilist = []
for k in range(len(i)):
    if u[k] > uni[i[k],j[k]]:
        uni[i[k],j[k]] = u[k]
        npk[i[k],j[k]] = n[k]
        for ii in range(3):
            for jj in range(3):
                ubifit[i[k],j[k],ii,jj] = ubi[ii,jj][k]
        ubilist.append(ubifit[i[k],j[k]].copy())

In [ ]:
def plot_inverse_pole_figure(UBs, ref_ucell, axis=np.array([0., 0, 1])):
    # get a meta orientation for all the grains
    meta_orien = ref_ucell.get_orix_orien(UBs)

    try:
        from orix.vector.vector3d import Vector3d
    except ImportError:
        raise ImportError("Missing diffpy and/or orix, can't compute orix phase!")

    ipf_direction = Vector3d(axis)

    # get the RGB colours
    rgb = ref_ucell.get_ipf_colour_from_orix_orien(meta_orien, axis=ipf_direction)

    # scatter the meta orientation using the colours
    meta_orien.scatter("ipf", c=rgb, direction=ipf_direction, s=1)

    
def plot_all_ipfs(UBs, ref_ucell):
    plot_inverse_pole_figure(UBs, ref_ucell, axis=np.array([1., 0, 0]))
    plot_inverse_pole_figure(UBs, ref_ucell, axis=np.array([0., 1, 0]))
    plot_inverse_pole_figure(UBs, ref_ucell, axis=np.array([0., 0, 1]))

In [ ]:
ublist = [np.linalg.inv(ubi) for ubi in ubilist]
ubarray = np.array(ublist)

plot_all_ipfs(ubarray, ref_ucell)

In [ ]:
def normalise( r ):
    negatives = r [r < 0 ]
    noise = np.std(negatives)
    s = ndi.uniform_filter( r, 5 )
    positives =  s > noise
    height = r[positives].mean()
    return np.where(s>noise, r/r.max(), 0)

In [ ]:
r = npk.copy()
ubisel = ubifit.copy()

In [ ]:
rgb = np.zeros( (3, ubisel.shape[0], ubisel.shape[1], 3 ))
for i in range(ubisel.shape[0]):
    for j in range(ubisel.shape[1]):
        for k,axis in enumerate( ((1,0,0),(0,1,0),(0,0,1)) ):
            if np.isnan( ubisel[i,j,0,0] ):
                rgb[k,i,j] = np.nan
            else:
                this_ubi = ubisel[i,j]
                this_ub = np.linalg.inv(this_ubi)
                rgb[k,i,j] = ref_ucell.get_ipf_colour(this_ub, axis)

In [ ]:
fig, a = plt.subplots(2,2,sharex=True,sharey=True, figsize=(10,10), constrained_layout=True)
a[0][0].imshow(rgb[0], origin="lower")
a[0][0].set_title("IPF X")
a[0][1].imshow(rgb[1], origin="lower")
a[0][1].set_title("IPF Y")
a[1][0].imshow(rgb[2], origin="lower")
a[1][0].set_title("IPF Z")
m = r > 0.02
a[1][1].imshow(np.where(m, r, np.nan), origin="lower")
a[1][1].set_title("Number of peaks")
# for ax in a.ravel():
#     ax.set(yticks=[], xticks=[])
#     ax.invert_yaxis()

plt.show()